# Ingeniería de características y Similitud sintáctica 

En este apartado se verá principalmente cómo *vectorizar* documentos, que consiste en convertir texto no estructurado en vectores compuestos por números.

Debido a que la vectorización es la base para casi todas las tareas de *Machine Learning*, en este capítulo se trabajará con dos modelos dispuestos por la librería *scikit-learn*, además de construir nuestro propio vectorizador, útil para futuros proyectos por ser ajustable a las tareas que en cada momento el usuario considere necesarias.

In [59]:
import sys, os

#Carga del archivo setup.py
%run -i ../pyenv_settings/setup.py

#Imports y configuraciones de gráficas
%run "$BASE_DIR/pyenv_settings/settings.py"

#Reset del entorno virtual al iniciar la ejecución
#%reset -f

%reload_ext autoreload
%autoreload 0
%config InlineBackend.figure_format = 'png'

# # to print output of all statements and not just the last
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# # otherwise text between $ signs will be interpreted as formula and printed in italic
# pd.set_option('display.html.use_mathjax', False)

You are working on a local system.
Files will be searched relative to "..".


Tras cargar los ajustes y preferencias del entorno virtual, podemos proceder a crear nuestro primer vectorizador.

## Construcción de un Vectorizador
Se va a construir un vectorizador que se usará sobre el Data Frame con el que se ha estado trabajando en los capítulos anteriores. Como primer paso, se deberá cargar el dataset guardado en la base de datos, cuya contenido (los comentarios de los usuarios) ya está normalizado y tokenizado.

In [60]:
#Conexión con la base de datos en la que tenemos guardado el Data Frame
db_name = "../data/zigbee2mqtt_comments.db"
con = sqlite3.connect(db_name)
df = pd.read_sql("select * from comments", con)
con.close()

#Comprobación de que se ha cargado correctamente
print(df.columns)
print(df[['normalized_text', 'tokens']].head())

Index(['id', 'user', 'text', 'impurity', 'clean_text', 'normalized_text',
       'tokens'],
      dtype='object')
                                                                                                                                                                                           normalized_text  \
0                                                                    This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days   
1  Also, after updating the z2m, cyclic reboots began ''' Starting Zigbee2MQTT without watchdog. INFO: Preparing to start... INFO: Socat not enabled INFO: Starting Zigbee2MQTT... Starting Zigbee2MQTT...   
2  Hi ! Since 2 or 3 days, MQTT suddenly fail. A few messages in the log, many auto restart, and works again ... Very strange. In the log INFO: Preparing to start... ERROR: Got unexpected response fr...   
3  I don't know if it's exactly the same, but since v1.42 I ha

### Enumeración del vocabulario
En primer lugar se van a enumerar todas las palabras de los comentarios normalizados del Data Frame (tokens), de este modo para referirnos a una palabra específica, se puede usar su número asociado (índice del diccionario generado) para crear los vectores.

*Antes que nada*, recordar que los tokens se almacenan en el Data Frame como palabras separadas por comas, no como una lista de palabras, por ello hay que convertir cada entrada de la columna "tokens" a listas que tengan cada palabra como una única entrada.

In [ ]:
#Asegurarse de que la columna 'tokens' contenga listas y no cadenas de texto
df['tokens'] = df['tokens'].apply(lambda x: x.split(','))

#Verificar cómo se ve ahora la columna 'tokens'
print(df['tokens'].head(5))

#Aunque parezca que no ha cambiado, ahora es precisamente una lista y se podrá crear
#el diccionario correctamente

0                                           [This, issue, is, stale, because, it, has, been, open, 30, days, with, no, activity, Remove, stale, label, or, comment, or, this, will, be, closed, in, 7, days]
1    [Also, after, updating, the, z2m, cyclic, reboots, began, Starting, Zigbee2MQTT, without, watchdog, INFO, Preparing, to, start, INFO, Socat, not, enabled, INFO, Starting, Zigbee2MQTT, Starting, Zi...
2    [Hi, Since, 2, or, 3, days, MQTT, suddenly, fail, A, few, messages, in, the, log, many, auto, restart, and, works, again, Very, strange, In, the, log, INFO, Preparing, to, start, ERROR, Got, unexp...
3    [I, don't, know, if, it's, exactly, the, same, but, since, v1.42, I, have, trouble, with, Z2M, It, restarts, x, times, a, day, without, further, notice, I, think, it, is, a, software, issue, becau...
4    [Hello, I, have, the, same, problem, see, the, log, file, zigbee2mqtt_2024-12-17T07-07-00.298Z.log, for, details, Everything, runs, normally, until, 00:37:59, after, which, no

In [62]:
#Creacion del diccionario a partir de la columna tokens ya existente
vocabulary = set([word for tokens in df['tokens'] for word in tokens])

In [63]:
#Enumeración de las palabras (tokens)
word_to_index = {word: i for i, word in enumerate(vocabulary)}

In [64]:
#Impresión del diccionario con sus índices creado
for word, i in word_to_index.items():
    print(f"'{word}': {i}")

'-right': 0
'Permit': 1
''configuration.yml': 2
'node:vm:306:38': 3
'2021-12-30T00:58:34.995Z': 4
'Brazilian': 5
'Supervisore': 6
'overhere': 7
''pump_2': 8
'2021-09-18': 9
'correspond': 10
''screen': 11
''OfficeMotionSensor1': 12
''24184': 13
'chksum:23e2': 14
'administrator': 15
'L4': 16
'dopo': 17
'listOnTimeout': 18
'172.30.33.1:8099': 19
'Fireware': 20
'main': 21
'page': 22
'1hr': 23
'slowing': 24
'2024.4.5': 25
'setValue': 26
'404': 27
'cleanup': 28
'avail': 29
'1641547258': 30
'--depth': 31
''Connection': 32
'-rw-r--r': 33
'generatedMessage': 34
'coordinators': 35
'7.2.2': 36
'ID_PART_ENTRY_SIZE': 37
'HASS_Zigbee2mqtt': 38
'2022-07-06': 39
'CF': 40
'fur': 41
'known_devices': 42
'man': 43
':35': 44
'node:events:634:26': 45
'xxx:xxxxx@xxxx-timescaledb': 46
''OfficeLight1': 47
'pci0000:00': 48
'SLZM': 49
'nevermind': 50
''TEST2': 51
'2021-11-18_16-26-47': 52
'PUTTY': 53
'0xdddddddddddddddd': 54
'Synology': 55
'backup': 56
'2024-01-22.17-21-47': 57
'23:26:54Received': 58
'aluminum':

'ovviamente': 4842
'far': 4843
'2023-08-02T08:12:20': 4844
'2024-01-29T00:54:14.041Z': 4845
'15:52:19MQTT': 4846
'Dockerfile': 4847
'4min': 4848
'follow': 4849
'bluetooth': 4850
'192.168.5.122:8888': 4851
'12:16:06': 4852
'facil': 4853
':260': 4854
'Silabs': 4855
'Application': 4856
'12:47:24': 4857
'concerned': 4858
'Moreover': 4859
':1254': 4860
'retention': 4861
':0.3984': 4862
'Kitchen': 4863
';)': 4864
'169': 4865
':20210708': 4866
'Retrieving': 4867
'2023-07-06T10:29:36.467Z': 4868
'usb-ITEAD_SONOFF_Zigbee_3.0_USB_Dongle_Plus_V2_20220706183154-if00': 4869
'safe': 4870
'#408': 4871
''8': 4872
'Checked': 4873
'Loads': 4874
'Nginx': 4875
'uninstalling': 4876
'69': 4877
':254': 4878
'lock_state': 4879
'2.1.3': 4880
'exclussions': 4881
'depends': 4882
'COMPORT': 4883
'intrestingly': 4884
'small': 4885
'14:12:06': 4886
'hidraw0': 4887
'description': 4888
'2023-02-02T20:18:34.886Z': 4889
'2022-10-03T21:35:25.563Z': 4890
'better': 4891
'loops': 4892
'sudo': 4893
'HAP': 4894
'halted': 489

Como se observa, el diccionario cuenta con un total de 12617 entradas. Ahora se incluirá una nueva columna en el Data Frame en el que se indicará el índice de cada token que aparece en cada entrada de texto (comentario) correspondiente.

In [66]:
#Se añaden los índices de los tokens a la columna 'token_index' del DF
df['token_index'] = df['tokens'].progress_apply(lambda tokens: [word_to_index.get(token, -1) for token in tokens])

100%|██████████| 2678/2678 [00:00<00:00, 45907.77it/s]


In [67]:
print(df[['normalized_text', 'tokens', 'token_index']].head())

                                                                                                                                                                                           normalized_text  \
0                                                                    This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days   
1  Also, after updating the z2m, cyclic reboots began ''' Starting Zigbee2MQTT without watchdog. INFO: Preparing to start... INFO: Socat not enabled INFO: Starting Zigbee2MQTT... Starting Zigbee2MQTT...   
2  Hi ! Since 2 or 3 days, MQTT suddenly fail. A few messages in the log, many auto restart, and works again ... Very strange. In the log INFO: Preparing to start... ERROR: Got unexpected response fr...   
3  I don't know if it's exactly the same, but since v1.42 I have trouble with Z2M. It restarts x times a day without further notice. I think it is a software issue, becaue in o

### Vectorización de documentos
Para comparar vectores se debe asegurar que todos cuentan con las mismas dimensiones, por ello se utiliza el mismo diccionario para todos.

Si un texto no contiene una palabra, se indica con un 0 en su posición, en caso contrario, se indica con un 1. Se deduce entonces que la longitud de los vectores será igual a la longitud del diccionario generado.

Ahora se definirá una función que codificará todos los textos en vectores:

In [68]:
def onehot_encode(text):
    return [1 if w in text else 0 for w in vocabulary]

In [69]:
#Generación de los vectores one-hot
onehot_vectors = [onehot_encode(text) for text in df['normalized_text']]

#Verificación de que se han codificado todas las entradas
print(f"Total de vectores generados: {len(onehot_vectors)}")
print(f"Total de entradas del Data Frame: {len(df)}")

print(df[['normalized_text']].head())

Total de vectores generados: 2678
Total de entradas del Data Frame: 2678
                                                                                                                                                                                           normalized_text
0                                                                    This issue is stale because it has been open 30 days with no activity. Remove stale label or comment or this will be closed in 7 days
1  Also, after updating the z2m, cyclic reboots began ''' Starting Zigbee2MQTT without watchdog. INFO: Preparing to start... INFO: Socat not enabled INFO: Starting Zigbee2MQTT... Starting Zigbee2MQTT...
2  Hi ! Since 2 or 3 days, MQTT suddenly fail. A few messages in the log, many auto restart, and works again ... Very strange. In the log INFO: Preparing to start... ERROR: Got unexpected response fr...
3  I don't know if it's exactly the same, but since v1.42 I have trouble with Z2M. It restarts x times a day withou

In [71]:
for text, vector in zip(df['normalized_text'].head(5), onehot_vectors):
    print("One-hot vector: ")
    print(vector)
    print(" - Normalized text: ")
    print(text)
    print("-" * 50)

One-hot vector: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0